# Text Detection Training


In [ ]:
ROOT_CONFIG_FOLDER = 'configs/textdet'
CONFIG_FOLDER_TO_CKPT = {
    'dbnetpp_custom': 'https://download.openmmlab.com/mmocr/textdet/dbnetpp/dbnetpp_resnet50-dcnv2_fpnc_1200e_icdar2015/dbnetpp_resnet50-dcnv2_fpnc_1200e_icdar2015_20220829_230108-f289bd20.pth',
    'dbnet_custom': 'https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_resnet50-dcnv2_fpnc_1200e_icdar2015/dbnet_resnet50-dcnv2_fpnc_1200e_icdar2015_20220825_221614-7c0e94f2.pth',
    'psenet_custom': 'https://download.openmmlab.com/mmocr/textdet/psenet/psenet_resnet50_fpnf_600e_icdar2015/psenet_resnet50_fpnf_600e_icdar2015_20220825_221459-7fe6b9cf.pth',
    'panet_custom': 'https://download.openmmlab.com/mmocr/textdet/panet/panet_resnet50_fpem-ffm_600e_icdar2015/panet_resnet50_fpem-ffm_600e_icdar2015_20220825_221459-7a8f4a5d.pth',
    'textsnake_custom': 'https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_resnet50_fpn-unet_1200e_ctw1500/textsnake_resnet50_fpn-unet_1200e_ctw1500_20220825_221459-c0b8c681.pth',
    'fcenet_custom': 'https://download.openmmlab.com/mmocr/textdet/fcenet/fcenet_resnet50-dcnv2_fpn_1500e_icdar2015/fcenet_resnet50-dcnv2_fpn_1500e_icdar2015_20220825_221510-4d705392.pth',
}


In [ ]:
# #@title Train single model

# from pathlib import Path
# from mmengine.runner import Runner
# import time
# from mmengine import Config

# %cd ~/bonting-identification

# model_config = 'configs/textdet/dbnetpp_custom/dbnetpp_custom_v0.py'
# # model_config = 'configs/textdet/dbnet_custom/dbnet_cegdr.py'
# # model_config = 'configs/textdet/psenet_custom/psenet_cegdr.py'
# # model_config = 'configs/textdet/panet_custom/panet_cegdr.py'
# # model_config = 'configs/textdet/textsnake_custom/textsnake_cegdr.py'
# # model_config = 'configs/textdet/fcenet_custom/fcenet_cegdr.py'

# cfg = Config.fromfile(Path(ROOT_CONFIG_FOLDER) / model_config)
# cfg['load_from'] = CONFIG_FOLDER_TO_CKPT[Path(model_config).parent.name]
# cfg.visualizer.name = f'{time.localtime()}'

# # Optionally, smoke test on 1 epoch
# # cfg.train_cfg['max_epochs'] = 1

# runner = Runner.from_cfg(cfg)
# result = runner.train()


In [ ]:
# RESULTS of model comparison on CEGD-R

# DBNet++ finetuned: cegdr/hmean: ???
# DBNet finetuned: cegdr/hmean: ???
# PSENet finetuned: cegdr/hmean: ???

In [ ]:
# !rm -rf work_dirs/*


In [ ]:
#@title Train all models

from mmengine.runner import Runner
import time
from mmengine import Config
import pandas as pd
from pathlib import Path

%cd ~/bonting-identification

results = []
model_configs = []
ckpts = []

for config_folder, model_url in list(CONFIG_FOLDER_TO_CKPT.items())[:1]:
    # Construct full path to config folder
    config_folder_path = Path(ROOT_CONFIG_FOLDER) / config_folder
    
    for model_config in [
        p for p in config_folder_path.glob('*.py')
        if not p.name.startswith('_base')
    ]:
        cfg = Config.fromfile(model_config)
        cfg['load_from'] = model_url
        cfg.visualizer.name = f'{time.localtime()}'

        cfg.train_cfg['max_epochs'] = 1

        runner = Runner.from_cfg(cfg)
        result = runner.train()

        results.append(result)
        model_configs.append(Path(model_config).name.rstrip('.py'))
        ckpts.append(Path(model_url).parts[-2])


In [ ]:
results_df = pd.DataFrame(results)
results_df.insert(0, 'model_config', model_configs)
results_df.insert(1, 'ckpt', ckpts)
results_df = results_df.set_index(['model_config', 'ckpt'])
results_df.sort_values('cegdr/hmean', ascending=False, inplace=True)
results_df


In [ ]:
save_path = Path('reports/eval/cegdr/textdet/mmocr_finetuned_det_results.csv')
save_path.parent.mkdir(parents=True, exist_ok=True)
print(f'Saving results to:\n{save_path}')
results_df.to_csv(save_path, index=True, header=True)
